# Pepperpot Emittance Calculations and Graphical Output

The code cell below is just to set up some packages and global variables so I can test code cells further down. Feel free to skip to another section.

In [1]:
import numpy as np
from scipy import constants as const
import math

def pep():

    #these will be used in calculating normalized emittance
    
    global e_amu
        
    e_amu = 931.5 # rest energy of 1 AMU in MeV
    
    global c
        
    c = float(const.c) # speed of light
    
    global e_rfq
    
    e_rfq = 0.0305 # RFQ acceptance energy 0.0305 MeV/u
              
    global v
    
    v = ((2*e_rfq)/e_amu)**0.5 * c # ion velocity in meters/second
    
    global lorentz_beta
    
    lorentz_beta = v/c # lorentz beta
    
    global lorentz_gamma
    
    lorentz_gamma = 1/((1-lorentz_beta**2)**0.5) # lorentz gamma

# User Input

Before running the code, we need to decide which physical pepperpot configuration we're using: the weak beam (CARIBU) setup, the ECR setup, or something else that should be entered manually. The main differences here will be the mask hole spacing and the distance from the mask to the phosphor. This in turn affects the resolution (mm/pixel) in the image procuced by the CCD. All of these are required for an effective calculation.

In [ ]:
    def user_prompt():

            try:
                choice = float(input('''Please select pepperpot configuration: 
                          1: Stable ion (ECR)
                          2: Weak beam (CARIBU)
                          3: User defined setup
                          
                          '''))
                if choice != 1 and choice != 2 and choice != 3:
                    raise Exception
                    
            except:
                print("Please select option 1, 2, or 3.")
                user_prompt()                

            if choice == 3: 
                L = float(input("Enter distance from mask to phosphor in mm: "))
                dh = float(input("Enter spacing between mask holes in mm: "))
                res_x = float(input("Enter image x-resolution in mm/pixel: "))
                res_y = float(input("Enter image y-resolution in mm/pixel: "))
                
            if choice == 1:
                L = 100.0 # distance from mask to phosphor in mm
                dh = 4 # spacing between mask holes in mm
                res_x = 0.136 # x resolution in mm/pixel
                res_y = 0.114 # y resolution in mm/pixel
                print("Stable Ion (ECR) Configuration")

            if choice == 2:
                L = 13.7 # distance from mask to phosphor in mm
                dh = 1.5 # spacing between mask holes in mm
                res_x = 0.0412 # x resolution in mm/pixel
                res_y = 0.0412 # y resolution in mm/pixel
                print("Weak Beam (CARIBU) Configuration")
                
            print(L)
                
user_prompt()



# Loading the Image Data

The image output arrays need to be loaded and this is also a good place to declare some global variables. We'll need the position of a pixel (or row/column of pixels, or other sample of the image, to be determined) in x and y coordinates, the associated pixel intensity, and the pixel moment--that is, how far the pixel is displaced from the center of the corresponding mask hole. The moment may be more easy to break down by finding the difference between the pixel and the unweighted (geometric) beamlet center, and the distance from the hole center to the beamlet center, and adding those two distances together. In the case of missing beamlets, this is also where the correction for that would go...something like, if the distance between a beamlet and the previous beamlet is more than 1.5 times the hole spacing then assign zero for the intermediate hole (and make a loop for 2.5 times the spacing, 3.5, etc., or find the nearest integer number of hole spaces or something like that...figure out later)

# Establishing the phase space components

*Note: from this point forward calculations are expressed in terms of $x$ but are easily adapted for calculations in $y$.*

Let a beamlet pixel have intensity $I_m$ and pixel center position $x_m$, and the corresponding mask hole center position $x_h$, with $L$ the distance from mask to phosphor. Then

\begin{equation}
    x^\prime \equiv \frac{x_m-x_h}{L}
\end{equation}

and

\begin{equation}
    I_{tot} \equiv \sum\limits_{m} I_m
\end{equation}

**Note:** $x^\prime$ is a measure of the angle between the z-axis (parallel to the beam pipe) and the ion trajectory. It takes advantage of the small angle approximation $\sin{a} \approx a$ where here the small value we're looking at is the distance in the x-direction or y-direction between the mask hole and the beamlet on the phosphor. Therefore it is important to have a way to identify the center of the screen and measure the x/y divergence of the beamlets.

Different institutions do this differently. A paper from SLAC shows calculation of $x^\prime$ from one row of pixels for each row of beamlets (corresponding to the most intensity peaks). CERN does one $x^\prime$ value for each individual beamlet. RIKEN does one for each pixel, and that produces the more impressive graphs of phase space components, while minimizing errors and not being particularly more difficult than the other methods.

To calculate the emittance, we need to extract the following phase space components from the image:

\begin{equation}
    \langle x \rangle = \frac{1}{I_{tot}} \sum\limits_{m} x_m I_m \\
\end{equation}

\begin{equation}
    \langle x^\prime \rangle = \frac{1}{I_{tot}} \sum\limits_{m} x^\prime_m I_m \\
\end{equation}

\begin{equation}
    \langle x^2 \rangle = \frac{1}{I_{tot}} \sum\limits_{m} (x_m - \langle x \rangle )^2 I_m \\
\end{equation}

\begin{equation}
    \langle x^{\prime 2} \rangle = \frac{1}{I_{tot}} \sum\limits_{m} (x_m^\prime - \langle x^\prime \rangle )^2 I_m \\
\end{equation}

\begin{equation}
    \langle x x^\prime \rangle = \frac{1}{I_{tot}} \sum\limits_{m} (x_m - \langle x \rangle ) (x_m^\prime - \langle x^\prime \rangle ) I_m \\
\end{equation}



In [3]:
    def phase_space_components():
        
        # for now I will call a pixel's intensity value I_pix, and the coordinates x_pos and y_pos and moments x_p and y_p since I don't know how the arrays will be structured.
        
        I_tot = np.sum(I_pix) # sum of all pixel intensities; this is used as a scaling factor for normalization.
        
        x_avg = (np.sum(x_pos*I_pix))/I_tot
        
        x_p_avg = (np.sum(x_p*I_pix))/I_tot
        
        x_sq_avg = (np.sum(((x_pos-x_avg)**2)*I_pix))/I_tot
        
        x_p_sq_avg = (np.sum(((x_p-x_p_avg)**2)*I_pix))/I_tot
        
        x_x_p_avg = (np.sum(((x_pos-x_avg)*(x_p-x_p_avg))*I_pix))/I_tot
        
        y_avg = (np.sum(y_pos*I_pix))/I_tot
        
        y_p_avg = (np.sum(y_p*I_pix))/I_tot
        
        y_sq_avg = (np.sum(((y_pos-y_avg)**2)*I_pix))/I_tot
        
        y_p_sq_avg = (np.sum(((y_p-y_p_avg)**2)*I_pix))/I_tot
        
        y_y_p_avg = (np.sum(((y_pos-y_avg)*(y_p-y_p_avg))*I_pix))/I_tot
        

In [ ]:
# It is possible that I am defining these incorrectly and I should not be using np.sum...

        x_avg_new = np.zeros(len(I_pix))
        
        for i in range(0, len(I_pix))
        
        x_avg_new[i] = (x_pos[i] * I_pix[i])
        
        x_avg_2 = np.sum(x_avg_new)/I_tot
        
        print(x_avg)
        print(x_avg_2)
        
# nope these are exactly the same and I also tested x_sq_avg with the same method


# Emittance and Twiss Parameters

Once the work in the previous section is done, the emittance and Twiss parameter calculations are fairly straightforward. The RMS (or geometric) emittance, $\epsilon_x$ is found as follows:

\begin{equation}
    \epsilon_x = \sqrt{\langle x^2 \rangle \langle x^{\prime 2} \rangle - \langle x x^\prime \rangle}
\end{equation}

From there, the Twiss parameters $\gamma$, $\beta$, and $\alpha$ can be calculated:

\begin{equation}
    \beta_x = \frac{\langle x^2 \rangle}{\epsilon_x}
\end{equation}

\begin{equation}
    \gamma_x = \frac{\langle x^{\prime 2} \rangle}{\epsilon_x}
\end{equation}

\begin{equation}
    \alpha_x = \sqrt{\beta_x \gamma_x - 1}
\end{equation}

If we know the ion velocity, we can then calculate the normalized emittance:

\begin{equation}
    \epsilon_{norm\, x} = 4 \gamma \beta \epsilon_x
\end{equation}

where $\gamma$ and $\beta$ are the Lorentzian $\gamma$ and $\beta$, not the Twiss parameters. Using the RFQ acceptance energy ($e_{rfq}$) of $30.5$ keV/u ($0.0305$ MeV/u), the rest energy of one amu which is $931.5$ MeV, and the speed of light $c$ we can combine these values to express the ion velocity $v$ (derived from $E = 1/2 mv^2$) as

\begin{equation}
    v = \sqrt{\frac{2\ast e_{rfq}}{e_{amu}}} c
\end{equation}


In [4]:
    def emittance_calc():
                
        eps_rms_x = math.sqrt((x_sq_avg*x_p_sq_avg-x_x_p_avg))
        
        eps_rms_y = math.sqrt((y_sq_avg*y_p_sq_avg-y_y_p_avg))
        
        eps_norm_x = 4*lorentz_gamma*lorentz_beta*eps_rms_x
        
        eps_norm_y = 4*lorentz_gamma*lorentz_beta*eps_rms_y
        
        twiss_beta_x = x_sq_avg/eps_rms_x
        
        twiss_gamma_x = x_p_sq_avg/eps_rms_x
        
        twiss_alpha_x = math.sqrt((twiss_beta_x*twiss_gamma_x-1))
        
        twiss_beta_y = y_sq_avg/eps_rms_y
        
        twiss_gamma_y = y_p_sq_avg/eps_rms_y
        
        twiss_alpha_y = math.sqrt((twiss_beta_y*twiss_gamma_y-1))
        

# Output Graphs

It would be useful to have some kind of graph--probably color heatmap--of the intensity on an x-y plot, x-x', y-y', and possibly x'-y' plot. However, attempting to develop this might not be very productive prior to having a set of test data and confirmation that the correct emittance values are being output.

# Tasks

Inspect physical pepperpot setup for mask to phosphor distances (done).

Update User Input section with powerpoint values from Brahim.

In [ ]:
# Here's a heatmap using the histogram2d function:

import numpy as np
import numpy.random
import matplotlib.pyplot as plt

# Create data
x = np.random.randn(4096)
y = np.random.randn(4096)

# Create heatmap
heatmap, xedges, yedges = np.histogram2d(x, y, bins=(64,64))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

# Plot heatmap
plt.clf()
plt.title('Pythonspot.com heatmap example')
plt.ylabel('y')
plt.xlabel('x')
plt.imshow(heatmap, extent=extent)
plt.show()